In [1]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, HfApiModel, load_tool, tool
import datetime
import requests
import pytz
import yaml

c:\Users\mathe\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import os
import re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Ollama

In [4]:
# Configurações
class Config:
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
    LLM_MODEL = "llama3.2:3b"
    TEMPERATURE = 0.5
    CHUNK_SIZE = 1500
    CHUNK_OVERLAP = 200
    PERSIST_DIR = "./chroma_db_pt_legal"
    PDF_PATHS = ["./docs/ctb.pdf", "./docs/mbvt20222.pdf"]

In [5]:
# Função de limpeza (mesma do código anterior)
def clean_brazilian_legal_text(text: str) -> str:
    patterns = [
        r"Diário Oficial .+? Página \d+",
        r"Lei Nº \d+\.\d+ de \d{2}/\d{2}/\d{4}",
        r"Publicado em: \d{2}/\d{2}/\d{4}",
        r"Este texto não substitui o original publicado",
        r"\n\s*\d+\s*\n"
    ]
    
    for pattern in patterns:
        text = re.sub(pattern, "", text, flags=re.IGNORECASE)
    
    text = re.sub(r"(?i)(artigo|art\.) ?(\d+)", r"Art. \2", text)
    text = re.sub(r"§ ?(único|\d+º?)", r"§ \1", text)
    
    return re.sub(r"\s+", " ", text).strip()

In [6]:
# Carregar e processar documentos
documents = []
for path in Config.PDF_PATHS:
    try:
        loader = PyPDFLoader(path)
        pages = loader.load_and_split(text_splitter=None)
        for page in pages:
            cleaned = clean_brazilian_legal_text(page.page_content)
            if cleaned.strip():
                page.page_content = cleaned
                documents.append(page)
        print(f"✓ {os.path.basename(path)} processado")
    except Exception as e:
        print(f"✗ Erro em {path}: {str(e)}")

✗ Erro em ./docs/ctb.pdf: File path ./docs/ctb.pdf is not a valid file or url
✗ Erro em ./docs/mbvt20222.pdf: File path ./docs/mbvt20222.pdf is not a valid file or url


In [7]:
# Configuração do text splitter jurídico corrigida
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=Config.CHUNK_SIZE,
    chunk_overlap=Config.CHUNK_OVERLAP,
    separators=[
        r"\n\nArt\. \d+\.",  # Separa por artigos
        r"\n§ ",             # Separa por parágrafos
        r"\n\n", 
        r"\n", 
        " ", 
        ""
    ],
    length_function=lambda x: len(x.split()),
    is_separator_regex=False  # Desativa regex complexo
)

In [8]:
# Processamento de documentos
texts = text_splitter.split_documents(documents)

# Verificação pós-split
print(f"Total de chunks gerados: {len(texts)}")
print("Exemplo de chunk inicial:")
print(texts[0].page_content[:500] + "...")

Total de chunks gerados: 0
Exemplo de chunk inicial:


IndexError: list index out of range

In [ ]:
# Verificar duplicatas
seen = set()
duplicates = 0
for t in texts:
    h = hash(t.page_content.strip().lower())
    if h in seen:
        duplicates += 1
    seen.add(h)
print(f"Chunks duplicados: {duplicates}")

In [ ]:
# Inicializar embeddings
embeddings = HuggingFaceEmbeddings(model_name=Config.EMBEDDING_MODEL)

In [ ]:
# Criar vetorstore
db = Chroma.from_documents(
    texts,
    embeddings,
    persist_directory=Config.PERSIST_DIR,
    collection_metadata={"hnsw:space": "cosine"}
)

In [ ]:
# Configurar retriever
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 7,
        "lambda_mult": 0.45,
        "score_threshold": 0.25
    }
)

In [ ]:
# Inicializar LLM
llm = Ollama(
    model=Config.LLM_MODEL,
    temperature=Config.TEMPERATURE,
    system="Você é um especialista em legislação de trânsito brasileira. Responda com base nos documentos fornecidos."
)

In [ ]:
# Prompt template
prompt_template = """
    Analise os seguintes trechos e responda de forma estruturada:
    
    Contexto:
    {context}
    
    Pergunta: {question}
    
    Inclua:
    1. Base legal (artigos/parágrafos)
    2. Explicação técnica
    3. Fontes (documento e página)
    
    Resposta:
"""

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
# Criar cadeia QA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
def consultar(pergunta):
    try:
        result = qa_chain({"query": pergunta})
        
        print(f"\n🔍 Pergunta: {pergunta}")
        print(f"\n📝 Resposta:\n{result['result']}")
        
        print("\n📚 Fontes:")
        seen_sources = set()
        for doc in result['source_documents']:
            src = f"{os.path.basename(doc.metadata['source'])} - Página {doc.metadata['page']}"
            if src not in seen_sources:
                print(f"  - {src}")
                seen_sources.add(src)
                
    except Exception as e:
        print(f"Erro: {str(e)}")

In [ ]:
# Exemplo 1
consultar("Qual é a idade mínima para habilitação na categoria D?")

In [ ]:
# Exemplo 2
consultar("Quais as penalidades por dirigir sob efeito de álcool?")

In [ ]:
# Exemplo 3
consultar("Compare as exigências para as categorias B e D")

In [ ]:
consultar(
    '''
    Escolha a opção correta: Para habilitar-se na categoria “D”, o candidato deverá ser: 
    a) maior de 18 anos; 
    b) penalmente imputável, não importando sua idade; 
    c) menor de 21 anos; 
    d) maior de 21 anos"
    '''
)

In [ ]:
consultar(
'''
    Escolha a opção correta: Para dirigir com segurança, evitando acidentes, o condutor deve demonstrar:
    
    Opção 1: habilidade ao dirigir; conhecimento das regras de trânsito; cooperação com os demais usuários da via;
    Opção 2: conhecimento de algumas regras de trânsito; agressividade nas situações perigosas; habilidade ao dirigir;
    Opção 3: bom senso; respeito apenas às regras mais importantes para a segurança; habilidade ao dirigir;
    Opção 4: habilidade ao dirigir; conhecimento de algumas regras de trânsito; bom senso.
    '''
)

In [ ]:
consultar(
    '''
    Escolha a opção correta: O condutor de veículo deve dar preferência de passagem aos pedestres:
    
    Opção 1: somente quando estão atravessando na faixa de pedestres;
    Opção 2: que não tenham concluído a travessia, quando houver mudança de sinal;
    Opção 3: caso as pessoas estejam próximas a área escolar;
    Opção 4: somente quando isso for solicitado pelo agente de trânsito.
    '''
)

<img src="https://portal.wemeds.com.br/wp-content/uploads/2023/04/escore-news.jpg" width="500"/>

In [ ]:
@tool
def clinical_deterioration_risk(resp_rate: int, spo2: int, bp_systolic: int, heart_rate: int, temp: float, avpu: str) -> str:
    """Calcula o risco de deterioração clínica com base no escore NEWS.
    
    Args:
        resp_rate: Frequência respiratória (rpm)
        spo2: Saturação de oxigênio (%)
        bp_systolic: Pressão arterial sistólica (mmHg)
        heart_rate: Frequência cardíaca (bpm)
        temp: Temperatura corporal (°C)
        avpu: Nível de consciência (Alerta, Responde à Voz, Responde à Dor, Inconsciente)
    
    Returns:
        Uma string indicando o nível de risco do paciente.
    """

    if resp_rate <= 8:
        resp_score = 3
    elif 9 <= resp_rate <= 11:
        resp_score = 1
    elif 12 <= resp_rate <= 20:
        resp_score = 0
    elif 21 <= resp_rate <= 24:
        resp_score = 2
    else:
        resp_score = 3
    
    if spo2 >= 96:
        spo2_score = 0
    elif 94 <= spo2 <= 95:
        spo2_score = 1
    elif 92 <= spo2 <= 93:
        spo2_score = 2
    else:
        spo2_score = 3
    
    if bp_systolic >= 110 and bp_systolic <= 219:
        bp_score = 0
    elif 100 <= bp_systolic < 110:
        bp_score = 1
    elif 90 <= bp_systolic < 100:
        bp_score = 2
    else:
        bp_score = 3
    
    if heart_rate <= 40:
        hr_score = 3
    elif 41 <= heart_rate <= 50:
        hr_score = 1
    elif 51 <= heart_rate <= 90:
        hr_score = 0
    elif 91 <= heart_rate <= 110:
        hr_score = 1
    elif 111 <= heart_rate <= 130:
        hr_score = 2
    else:
        hr_score = 3
    
    if temp < 35:
        temp_score = 3
    elif 35 <= temp <= 36:
        temp_score = 1
    elif 36.1 <= temp <= 38:
        temp_score = 0
    elif 38.1 <= temp <= 39:
        temp_score = 1
    else:
        temp_score = 2
    
    avpu_score = 0 if avpu.lower() == "alerta" else 3
    total_score = resp_score + spo2_score + bp_score + hr_score + temp_score + avpu_score
    
    if total_score == 0:
        return "Risco mínimo - Monitoramento de rotina"
    elif 1 <= total_score <= 4:
        return "Risco baixo - Observação clínica"
    elif 5 <= total_score <= 6:
        return "Risco moderado - Revisão urgente por profissional de saúde"
    else:
        return "Risco alto - Atendimento médico imediato"